In [ ]:
import tensorflow as tf
tf.set_random_seed(7)
print ("TensorFlow version: " + tf.__version__)
import keras

In [ ]:
from mlxtend.data import loadlocal_mnist
from sklearn.model_selection import train_test_split
import numpy as np

files = {
    "X": "data/train-images-idx3-ubyte",
    "y": "data/train-labels-idx1-ubyte",
    "X_test": "data/t10k-images-idx3-ubyte",
    "y_test": "data/t10k-labels-idx1-ubyte"
}

# Code to load the binary files.
# https://github.com/rasbt/mlxtend/blob/master/mlxtend/data/local_mnist.py

X, y = loadlocal_mnist(images_path = files['X'], labels_path = files['y'])
X_test, y_test = loadlocal_mnist(images_path = files['X_test'], labels_path = files['y_test'])

### Reshape the data

In [ ]:
X = X.reshape(X.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)


# Split the data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=42)

print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)

### One-hot Encoding

In [ ]:
import numpy as np

def one_hot_encode(data):
    encoded_data = np.zeros((data.shape[0], 10), dtype=np.float32)
    for index, digit in enumerate(data):
        encoded_data[(index, digit)] = 1
    return encoded_data

    
encoded_y_train = one_hot_encode(y_train)
encoded_y_valid = one_hot_encode(y_valid)
encoded_y_test = one_hot_encode(y_test)

In [ ]:
# from keras.utils import to_categorical

# encoded_y_train = to_categorical(y_train)
# encoded_y_valid = to_categorical(y_valid)

In [ ]:
encoded_y_train[:10]

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten

# create model
model = Sequential()
# add model layers
model.add(Conv2D(64, kernel_size = 3, activation = 'relu', input_shape =(28, 28, 1)))
model.add(Conv2D(32, kernel_size = 3, activation = 'relu'))
model.add(Flatten())
model.add(Dense(10, activation = 'softmax'))

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model.fit(X_train, encoded_y_train, validation_data = (X_valid, encoded_y_valid), epochs = 3)

In [ ]:
from sklearn.metrics import accuracy_score

# y_test = to_categorical(y_test)

y_pred = model.predict(X_test)
# accuracy_score(y_test, y_pred)

In [ ]:
y_pred[:10]
op = np.unravel_index(np.argmax(y_pred, axis=1), y_pred.shape)

In [ ]:
accuracy_score(y_test, op[1])